In [1]:

!pip install geopy
!pip install folium
print("Installed!")

Installed!


In [2]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)[0]

In [4]:
# clean dataframe 
df_list = df_list[df_list.Borough!='Not assigned']
df_list = df_list[df_list.Borough!=0]
df_list.reset_index(drop = True, inplace = True)

i = 0
for i in range(0,df_list.shape[0]):
    if df_list.iloc[i][2] == 'Not assigned':
        df_list.iloc[i][2] = df_list.iloc[i][1]
        i = i+1